# ===========================================
# 📌 **Домашнее задание: Этическая классификация токсичности комментариев**
# Задания 1–10. **Датасет: Civil Comments**
# ===========================================

In [1]:
# 🛠 Установка библиотек (если требуется)
!pip install -q scikit-learn pandas numpy

In [2]:
# 📚 Импорт библиотек
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# ===========================
# 📦Загрузка и подготовка данных
# (для всех заданий используется этот блок)
# ===========================

In [3]:
# 🎲 Фиксируем random seed для воспроизводимости
np.random.seed(0)

In [5]:
# 🔽 Загрузка CSV-файла
data = pd.read_csv("/content/sample_data/data (1).csv")

In [6]:
# 🧾 Выделим колонки с комментариями и метками токсичности
comments = data["comment_text"]
target = (data["target"] > 0.7).astype(int)  # 1, если токсичный, иначе 0


# ===========================================
# ✅ Задание 1 — Разделение на train/test
# ===========================================

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    comments, target, test_size=0.3, random_state=0
)

# ==============================
# ✅ Задание 2 — Преобразование текста в числа
# ==============================

In [8]:
# Используем CountVectorizer для векторизации текста
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


# ===========================================
# ✅ Задание 3 — Обучение логистической регрессии
# ===========================================

In [9]:
classifier = LogisticRegression(max_iter=2000)
classifier.fit(X_train_vec, y_train)

LogisticRegression(max_iter=2000)

In [10]:
# Оценим точность модели
y_pred = classifier.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)
print(f"\n🎯 Accuracy модели: {accuracy:.4f}")


🎯 Accuracy модели: 0.9285


# ================================
# ✅ Задание 4 — Функция предсказания токсичности
# ================================

In [11]:
def predict_toxicity(comment):
    """Функция возвращает вероятность токсичности комментария"""
    vec = vectorizer.transform([comment])
    proba = classifier.predict_proba(vec)[0, 1]
    return proba

# ===============================
# ✅ Задание 5 — Проверка на "Apples are stupid" и "I love apples"
# ===============================

In [12]:
print("\n🍏 Проверка предсказаний:")
print(f"'Apples are stupid' → {predict_toxicity('Apples are stupid'):.4f}")
print(f"'I love apples' → {predict_toxicity('I love apples'):.4f}")


🍏 Проверка предсказаний:
'Apples are stupid' → 0.9995
'I love apples' → 0.0986


# ===========================================
# ✅ Задание 6 — 10 самых токсичных слов и коэффициенты
# ===========================================

In [13]:
# Получаем список всех слов (признаков)
feature_names = np.array(vectorizer.get_feature_names_out())
coefficients = classifier.coef_[0]  # Веса модели

In [14]:
# Индексы 10 слов с наибольшими коэффициентами
top10_idx = np.argsort(coefficients)[-10:][::-1]

In [15]:
print("\n🔥 Топ-10 токсичных слов и их веса:")
for idx in top10_idx:
    print(f"{feature_names[idx]}: {coefficients[idx]:.4f}")


🔥 Топ-10 токсичных слов и их веса:
stupid: 9.5496
idiot: 8.6762
idiots: 8.6292
stupidity: 7.5847
idiotic: 6.8082
crap: 6.5476
dumb: 6.5281
pathetic: 6.4570
morons: 6.3474
moron: 6.3433


# ===============================
# ✅ Задание 7 — Рефлексия
# ===============================
#
# ➤ Какие из слов вас удивили? Какие не должны считаться токсичными?
# ➤ Это может указывать на ошибку в обучающем датасете или контексте.

### Результаты в целом логичны: все представленные слова являются выражениями с ярко негативной окраской, направленной на унижение собеседника или других людей. Эти слова носят либо оскорбительный, либо пренебрежительный характер. Однако при этом у меня возникли некоторые мысли:

### **Ожидаемые понятия:**

* Слова вроде idiot, moron, dumb, stupid — это типичные примеры прямой вербальной агрессии. Их высокая токсичность очевидна и подтверждается их частым использованием в интернет-среде в оскорбительном контексте.

* Такие слова, как crap и pathetic, тоже логично оказались в списке: они часто используются в обесценивании мнения или личности оппонента.

### **Интересные наблюдения**:

* Все слова в списке являются универсально негативными и, скорее всего, используются как личностные оценки, а не как описание темы или явления. Это подчёркивает, что модель хорошо уловила именно лингвистические маркеры токсичности.

### **Что могло бы вызывать беспокойство:**

* В данном списке нет слов, связанных с темами религии, этничности, пола и т.д. — и это хорошо. Однако если бы в этот топ попали слова вроде muslim, jew, black, gay и др., я бы задумалась о наличии исторической или социальной предвзятости модели.

* Тем не менее, в других примерах (задание 8), мы уже наблюдали, что слова, связанные с религией или расой, могут вызывать искажения в классификации. Следовательно, сама модель может демонстрировать предвзятость — просто не обязательно через топ-10 слов.

### **Вывод:**
* Слова, вошедшие в топ-10, подтверждают, что модель научилась «распознавать» прямую токсичную лексику. Однако её способность учитывать контекст и различать оскорбления от нейтрального употребления нуждается в дальнейшем улучшении. Кроме того, важно дополнительно проверять модель на скрытую предвзятость, особенно по чувствительным категориям.



# ========================
# ✅ Задание 8 — Тест на bias модели
# ========================

In [16]:


print("\n🧪 Тест на возможную предвзятость (bias):")
bias_test_comments = [
    "I have a christian friend",
    "I have a muslim friend",
    "I have a white friend",
    "I have a black friend"
]

for comment in bias_test_comments:
    print(f"{comment} → {predict_toxicity(comment):.4f}")

# ===========================================


🧪 Тест на возможную предвзятость (bias):
I have a christian friend → 0.1217
I have a muslim friend → 0.4446
I have a white friend → 0.3270
I have a black friend → 0.5155


# =============================
 # ✅ Задание 9 — Тип предвзятости (комментарий)
# =============================




Более подробный ответ

###  В рамках теста на предвзятость я провела проверку модели на четырёх конструкциях, которые по своей сути являются нейтральными:

* "I have a christian friend" → 0.1217
* "I have a muslim friend" → 0.4446
* "I have a white friend" → 0.3270
* "I have a black friend" → 0.5155

### **💬 Мои наблюдения:**
* Семантически все фразы идентичны — они выражают наличие друга определённой религии или расы, без какой-либо негативной оценки. Однако предсказания модели варьируются достаточно сильно.

* Наиболее "токсичной" модель сочла фразу "I have a black friend" (0.5155) — вероятность токсичности выше 0.5, что может привести к её ошибочной классификации как токсичной.

* Фраза "I have a muslim friend" также получила сравнительно высокую вероятность токсичности (0.4446), особенно в контексте того, что аналогичная фраза про "christian friend" оценена значительно ниже (0.1217).

* Фраза с “white friend” получила промежуточное значение (0.3270), но тоже выше, чем для “christian friend”.

### **💬 Выводы:**
* Модель демонстрирует признаки социальной и исторической предвзятости, унаследованной от распределения данных в оригинальном датасете. Скорее всего, в обучающем корпусе слова muslim, black и white встречались в токсичных контекстах чаще, чем christian, из-за чего они получили более высокие веса в классификации.

* Это яркий пример того, как социальные предубеждения, присутствующие в обществе и отражённые в данных, транслируются в алгоритмы, даже если мы не задавали никаких явных параметров, связанных с расой или религией. Такая модель может в будущем:

* ошибочно маркировать нейтральные комментарии как токсичные;
усиливать дискриминацию; снижать доверие к ИИ-инструментам в чувствительных сферах.

### **💬 Этика и решение:**

* Важно учитывать подобные искажения при построении и внедрении моделей ИИ. Даже простые baseline-модели, как эта, уже могут неосознанно воспроизводить и закреплять социальные предрассудки. Чтобы минимизировать bias, необходимо:
* применять методы дебиасинга при подготовке данных;
* использовать сбалансированные датасеты;
* анализировать поведение модели на чувствительных категориях;
* привлекать специалистов по этике на этапах разработки.

# ===========================================
# ✅ Задание 10 — Идеи по улучшению этичности модели
# ===========================================

In [18]:

print("\n💡 Идеи по улучшению модели:")
print("1. Исключить из датасета чувствительные признаки (религия, раса и т.п.) или уравновесить их.")
print("2. Применить дебиасинг словарей — пересчитать веса для чувствительных токенов.")
print("3. Использовать интерпретируемые методы (например, LIME, SHAP) для аудита модели.")


💡 Идеи по улучшению модели:
1. Исключить из датасета чувствительные признаки (религия, раса и т.п.) или уравновесить их.
2. Применить дебиасинг словарей — пересчитать веса для чувствительных токенов.
3. Использовать интерпретируемые методы (например, LIME, SHAP) для аудита модели.
